In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.keys import Keys
import pyautogui
import pyperclip

In [ ]:
URL = 'https://page.kakao.com/menu/10/screen/5'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

week = ['tab_uid=1', 'tab_uid=2', 'tab_uid=3', 'tab_uid=4', 'tab_uid=5', 'tab_uid=6', 'tab_uid=7', 'tab_uid=12']
ko_week = ['월', '화', '수', '목', '금', '토', '일', '완결']

In [ ]:
id_list = [] ; title_list = [] ; author_list = [] ; day_list = [] ; genre_list = [] ; 
story_list = [] ; platform_list = [] ; img_list = [] ; tag_list = [] ; 
grade_list = [] ; viewer_list = [] ; series_list = [] ; regist_list = [] ; href_list = []
num = 0

In [ ]:
driver = webdriver.Chrome() # 크롬 사용하니까, 크롬 드라이버를 현재 파이썬 코드가 있는 폴더로!!

def kakao_login():
    login = 'https://accounts.kakao.com/login/?continue=https%3A%2F%2Fkauth.kakao.com%2Foauth%2Fauthorize%3Fis_popup%3Dfalse%26ka%3Dsdk%252F2.1.0%2520os%252Fjavascript%2520sdk_type%252Fjavascript%2520lang%252Fko-KR%2520device%252FWin32%2520origin%252Fhttps%25253A%25252F%25252Fpage.kakao.com%26auth_tran_id%3DnecNHSOBn6sHjPBQmiu8-fKVxYHZtyhD5HWb.32SrPLXkikjn9zPDUerpMUg%26response_type%3Dcode%26state%3Dhttps%25253A%25252F%25252Fpage.kakao.com%25252F%26redirect_uri%3Dhttps%253A%252F%252Fpage.kakao.com%252Frelay%252Flogin%26through_account%3Dtrue%26client_id%3D49bbb48c5fdb0199e5da1b89de359484&talk_login=hidden#login'
    driver.implicitly_wait(2)
    driver.maximize_window()
    driver.get(login)
    
#     driver.find_element_by_css_selector("#id_email_2_label").click()
    driver.find_element_by_xpath('//*[@id="loginKey--1"]').click()
    pyperclip.copy('{아이디}')
    pyautogui.hotkey('ctrl','v')
    sleep(5)
    
#     driver.find_element_by_css_selector('#id_password_3_label').click()
    driver.find_element_by_xpath('//*[@id="password--2"]').click()
    pyperclip.copy('{비번}')
    pyautogui.hotkey('ctrl','v')
    sleep(5)

    #로그인 버튼 
#     login_send = browser.find_element_by_css_selector("#login-form > fieldset > div.wrap_btn > button.btn_g.btn_confirm.submit")
    login_send = driver.find_element_by_xpath('//*[@id="mainContent"]/div/div/form/div[4]/button[1]')
    login_send.click()
#     driver.back()

In [ ]:
driver.get(URL)
driver.implicitly_wait(3) #로딩이 끝날 때까지 10초까지는 기다려줌
kakao_login() #로그인 

In [ ]:
for i in range(8):
#     sleep(0.5)
    uri = URL + "?" + week[i]

    driver.get(uri) #요일별로 링크 가져옴

    html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
    soup = BeautifulSoup(html, 'html.parser')
    
    sleep(0.5)

    #스크롤 전 높이
    before_h = driver.execute_script("return window.scrollY")

    #무한 스크롤
    while True:
        #맨 아래로 스크롤을 내린다
        driver.find_element_by_css_selector("body").send_keys(Keys.END)

        #스크롤 사이 페이지 로딩 시간
        sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        if after_h == before_h:
            break

        before_h = after_h

    pages = driver.find_elements_by_css_selector('#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col > div.flex.grow.flex-col > div > div:nth-child(3) > div > div > div > div > a')

    page_list = []
    
    for page in pages:
        temp = page.get_attribute('href')
        page_list.append(temp)

    for href in page_list:
    
        temp = href + "?tab_type=about"
    
        print(href)
    
        driver.get(temp)
    
        html = requests.get(temp).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
        soup = BeautifulSoup(html, 'html.parser')
    
        sleep(1)  # 2->1
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/button').click()
        except:
            pass
    
        # 작품 정보 저장    
        try:
            img = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[1]/div/div/img')
        except:
            driver.back()
            continue
            
        ### 제목이 이미 있으면 다음으로
        title = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/span').text
        if(title in title_list):
            driver.back()
            continue
        
        thumb = img.get_attribute('src').replace("&filename=th3", "")
        img_list.append(thumb)
    
        day = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/div[2]/div[1]/span').text
        day = day.replace(" 연재", "")
        day_list.append(day)
        
        #title = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/span').text
        id_list.append(num) ; num += 1
        title_list.append(title)
        href_list.append(href)
    
        #day = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/div[2]/div[1]/span').text
        #print(day.split(' ')[0])

        genre = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/div[1]/span[4]').text
        genre_list.append(genre)

        author = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/div[2]/div[2]/span').text
        author_list.append(author)
        
        ### 태그
        try: # tag 정보가 있을 경우
            tag_info = driver.find_elements_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/a/button')
            str = ""
            for info in tag_info:
                tag = info.text.replace("#", "")
                str += tag + " "
            print(str)
            if str == "":
                tag_list.append(genre)
            else:
                tag_list.append(str)
        except: # tag 정보가 없을 경우
            tag_list.append(genre)
#             tag_list.append("")        

        ### 스토리
        try:
            story = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[3]/span').text
        except:
            story = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/span').text
    
        story_list.append(story)
        
        ### 독자 수
        viewers = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/div[1]/span[1]').text
        viewer_list.append(viewers)
        
        ### 평점
        grade = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/div[3]/div[2]/div[1]/span[2]').text
        grade_list.append(grade)
        
#         sleep(2)

        ### 전체 회차
        driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/a').click()
        sleep(0.5)  
        
        try:
            series = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/span').text
        except:
            try:
                series = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/span').text
            except:
                series = "전체 0"
        
        series = series.replace("전체 ", "")
        print(series)
        series_list.append(series)
        
        sleep(0.5)
                                                
        ### 등록일
        try:
            state = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/span').text
        except:
            state = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div/span').text
        if state != "첫화부터":
            driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]').click()
            try:
                driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[1]/div[3]/div[1]/div').click()
            except:
                driver.refresh()
                driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]').click()
                driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[1]/div[3]/div[1]/div').click()
                                                
        first = driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/ul/li[1]/div/div/a/div/div[2]/div[2]/span[1]').text
        regist_list.append(first)

        ### 플랫폼
        platform_list.append("카카오페이지")
        
        sleep(1) 
        
        driver.back()
    
#         sleep(0.5)
    
        driver.back() # 뒤로 가기

In [ ]:
total_data = pd.DataFrame()
total_data['id'] = id_list
total_data['title'] = title_list
total_data['img'] = img_list ## 썸네일
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['tag'] = tag_list
total_data['viewer'] = viewer_list
total_data['grade'] = grade_list
total_data['regist'] = regist_list
total_data['series'] = series_list
total_data['href'] = href_list
total_data['platform'] = platform_list
total_data.to_csv('2_href_new_kakao_webtoon.csv', encoding = 'utf-8-sig')